In [9]:
import pandas as pd
import requests
from yahoofinancials import YahooFinancials
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
from sklearn.metrics import mean_absolute_error
import time

In [10]:
df_empresas = pd.read_excel('empresas.xlsx')

In [11]:
df_empresas.head()

,Simbolo,Nome_resumido,Nome_Completo,Setor,Resumo_Empresarial,Cidade,UF,website,Negocio,Valor_Empresa,Valor_Mercado,logo_url
0,ABEV3.SA,AMBEV S/A ON,Ambev,Consumer Defensive,"Ambev S.A., through its subsidiaries, produces...",São Paulo,SP,https://www.ambev.com.br,BeveragesBrewers,2140000000000,2210000000000,https://logo.clearbit.com/ambev.com.br
1,AFLT3.SA,AFLUENTE T ON,Afluente Transmissão de Energia Elétrica,Utilities,Afluente Transmissão de Energia Elétrica S.A. ...,Rio de Janeiro,RJ,https://www.afluentet.com.br,UtilitiesRegulated Electric,5779159680,5778585600,https://logo.clearbit.com/afluentet.com.br
2,AGRO3.SA,BRASILAGRO ON NM,BrasilAgro - Companhia Brasileira de Proprieda...,Consumer Defensive,BrasilAgro - Companhia Brasileira de Proprieda...,São Paulo,SP,https://www.brasil-agro.com,Farm Products,32589214720,30491591680,https://logo.clearbit.com/brasil-agro.com
3,ALPA4.SA,ALPARGATAS PN N1,Alpargatas,Consumer Cyclical,Alpargatas S.A. engages in the manufacture and...,São Paulo,SP,https://www.alpargatas.com.br,Footwear & Accessories,111000000000,111000000000,https://logo.clearbit.com/alpargatas.com.br
4,ALUP11.SA,ALUPAR UNT N2,Alupar Investimento,Utilities,"Alupar Investimento S.A., through its subsidia...",São Paulo,SP,https://www.alupar.com.br,UtilitiesRenewable,200000000000,162000000000,https://logo.clearbit.com/alupar.com.br


In [42]:
selecao = 'Alpargatas'

In [43]:
df_filtro = df_empresas.query(f"Nome_Completo == '{selecao}'")
df_filtro

,Simbolo,Nome_resumido,Nome_Completo,Setor,Resumo_Empresarial,Cidade,UF,website,Negocio,Valor_Empresa,Valor_Mercado,logo_url
3,ALPA4.SA,ALPARGATAS PN N1,Alpargatas,Consumer Cyclical,Alpargatas S.A. engages in the manufacture and...,São Paulo,SP,https://www.alpargatas.com.br,Footwear & Accessories,111000000000,111000000000,https://logo.clearbit.com/alpargatas.com.br


In [44]:
simbolo = list(df_filtro['Simbolo'])
simbolo

['ALPA4.SA']

In [48]:
def modelo_consolidado(simbolo):
    now = datetime.now()
    delta = timedelta(days=45)
    result = now - delta
    result = pd.to_datetime(result)
    new_df = yf.download(simbolo, start='2015-01-01',end=result)['Close']
    new_df.dropna(inplace=True)
    new_df = new_df.reset_index()
    valorz = np.load('arrays.npy')

    def monte_carlo_previsao(dataset, ativo, dias_a_frente,valorz):
        dataset = dataset.copy()
        dataset = pd.DataFrame(dataset[ativo])
        dataset_normalizado = dataset.copy()
        for i in dataset:
            dataset_normalizado[i] = dataset[i] / dataset[i][0]
        dataset_taxa_retorno = np.log(1 + dataset_normalizado.pct_change())
        dataset_taxa_retorno.fillna(0, inplace=True)
        media = dataset_taxa_retorno.mean()
        variancia = dataset_taxa_retorno.var()
        drift = media - (0.5 * variancia)
        desvio_padrao = dataset_taxa_retorno.std()
        retornos_diarios = np.exp(drift.values + desvio_padrao.values * valorz)
        previsoes = np.zeros_like(retornos_diarios)
        previsoes[0] = dataset.iloc[-1]
        for dia in range(1, dias_a_frente):
            previsoes[dia] = previsoes[dia - 1] * retornos_diarios[dia] 
        return previsoes.T
    
    gera_previsoes = monte_carlo_previsao(new_df, 'Close', 120, valorz)
    simu_monte_carlo = pd.DataFrame(gera_previsoes.T)
    corte = pd.to_datetime(new_df['Date'].tail(1).values)
    now = datetime.now()
    df_conf =yf.download(simbolo, start=corte[0],end=now)['Close']
    df_conf.dropna(inplace=True)
    df_conf = df_conf.reset_index()
    df_comparacao = pd.concat([df_conf, simu_monte_carlo.head(len(df_conf))], axis = 1)
    df_comparacao.rename(columns={'Close':'Preço_Real', 0:'Cenario-01',1:'Cenario-02',2:'Cenario-03',
                                  3:'Cenario-04',4:'Cenario-05'}, inplace=True)
    df_acoes =yf.download(simbolo, start='2017-01-01')['Close']
    df_acoes.dropna(inplace=True)
    df_acoes = df_acoes.reset_index()
    previsoes_final = monte_carlo_previsao(df_acoes, 'Close', 120, valorz)
    previsoes_final = pd.DataFrame(previsoes_final.T)
    start_gen_date = df_acoes['Date'].iloc[-1]
    num_days = 365
    dates = [start_gen_date + timedelta(days=x) for x in range(num_days)]
    df_datas = pd.DataFrame({'Date': dates})
    df_datas = df_datas[df_datas['Date'].dt.weekday < 5]
    df_datas = df_datas.query("Date not in ['2023-09-07','2023-10-12','2023-11-03','2023-11-15',\
                            '2023-12-25','2024-01-01']")
    df_datas = df_datas.sort_values(by='Date')
    df_datas = df_datas.head(len(previsoes_final))
    df_datas['Date'] = df_datas['Date'].dt.strftime('%Y-%m-%d')
    df_datas = df_datas.reset_index()
    df_final = pd.concat([df_datas,previsoes_final], axis = 1)
    df_final.rename(columns={0:'Cenario-01',1:'Cenario-02',2:'Cenario-03',
                            3:'Cenario-04',4:'Cenario-05'}, inplace=True)
    df_final['Date'] = pd.to_datetime(df_final['Date'])
    df_final = df_final.sort_values(by='Date')
    df_final.drop(columns=['index'], inplace=True)
    colu_cenarios = ['Cenario-01','Cenario-02','Cenario-03','Cenario-04','Cenario-05']
    dados_metricas = []
    for x in colu_cenarios:
        medida = np.sum(abs(df_comparacao[x] - df_comparacao['Preço_Real'])) / len(df_comparacao)
        dados_metricas.append([medida,x])
    df_mae = pd.DataFrame(dados_metricas).rename(columns={0:'MAE',1:'Cenario'}).round(2)       
        
    return df_final, df_mae

In [55]:
df_final, df_mae = modelo_consolidado(simbolo)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [56]:
df_mae

,MAE,Cenario
0,0.85,Cenario-01
1,0.98,Cenario-02
2,0.37,Cenario-03
3,0.55,Cenario-04
4,0.70,Cenario-05


In [41]:
df_final

,Date,Cenario-01,Cenario-02,Cenario-03,Cenario-04,Cenario-05
0,2023-05-17,9.000000,9.000000,9.000000,9.000000,9.000000
1,2023-05-18,8.563706,9.205688,9.057968,8.985563,8.223374
2,2023-05-19,9.391114,9.707416,8.961778,9.310566,8.461528
3,2023-05-22,9.855943,10.312025,8.615513,8.711677,7.958106
4,2023-05-23,10.380010,10.715041,8.436334,9.319977,7.344936
...,...,...,...,...,...,...
115,2023-10-27,11.214202,5.983799,6.805406,10.803438,4.078614
116,2023-10-30,10.896126,6.017645,6.474094,11.772154,3.831297
117,2023-10-31,10.491523,5.613937,6.283355,11.261352,4.090657
118,2023-11-01,11.225493,5.702163,6.195022,11.525450,4.058960


In [ ]:
mae1

In [ ]:
mae2

In [79]:

now = datetime.now()
delta = timedelta(days=45)
result = str(now - delta)
result = result[0:10]

In [67]:
new_df = yf.download(simbolo, start='2015-01-01',end=datetime.now() - timedelta (45))['Close']
new_df.dropna(inplace=True)
new_df = new_df.reset_index()

[*********************100%***********************]  1 of 1 completed


In [73]:
pd.to_datetime(datetime.now() - timedelta (45),format='%Y-%m-%d')

Timestamp('2023-04-03 12:32:53.524556')

In [68]:
new_df.tail(1)

,Date,Close
2051,2023-04-03,7.88


In [80]:
new_df.query(f"Date <= {result}")

SyntaxError: invalid syntax (<unknown>, line 1)